In [1]:
import numpy as np
import xarray as xr
from datetime import datetime, timedelta


In [2]:
def clip_nc_region(nc_file,date):
    # 打开 NetCDF 文件
    data = xr.open_dataset(nc_file)
    # 确定经纬度范围
    lon_range = np.linspace(112.0, 117.1, data['u'].shape[2])
    lat_range = np.linspace(12.9, 15.7, data['u'].shape[1])
    u = xr.DataArray(data['u'].values, dims=['lv', 'latu', 'lonu'], coords={'lonu': lon_range, 'latu': lat_range})
    
    # 创建 DataArray 对象并赋予坐标
    lon_range = np.linspace(112.0, 117.1, data['v'].shape[2])
    lat_range = np.linspace(12.9, 15.7, data['v'].shape[1])
    v = xr.DataArray(data['v'].values, dims=['lv', 'latv', 'lonv'], coords={'lonv': lon_range, 'latv': lat_range})
    
    # 确定目标经纬度范围
    target_lon = data['lon'].values
    target_lat = data['lat'].values
    
    # 进行插值
    regrid_u = u.interp(lonu=target_lon, latu=target_lat, method='linear')
    regrid_v = v.interp(lonv=target_lon, latv=target_lat, method='linear')
    
    # 截取 s, t, zeta 使用的 lon 和 lat
    # 指定要截取的经纬度范围
    lon_min, lon_max = 112, 117.1  # 经度范围
    lat_min, lat_max = 12.9, 15.7  # 纬度范围
    subset_t = data['t'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_s = data['s'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_zeta = data['zeta'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_u = regrid_u.sel(lonu=slice(lon_min, lon_max), latu=slice(lat_min, lat_max))
    subset_v = regrid_v.sel(lonv=slice(lon_min, lon_max), latv=slice(lat_min, lat_max))
    # 创建新的数据集
    subset = xr.Dataset({
        't': subset_t,
        's': subset_s,
        'zeta': subset_zeta,
        'u': subset_u,
        'v': subset_v
    })
    #保存为新的 NetCDF 文件
    subset.to_netcdf('E:/DataSet/redos/Subset_1.0_1995/subset_'+date+'.nc')

In [4]:
def date_incrementer(start_year, end_year):
    # 从指定年份的1月1日开始
    current_date = datetime(start_year, 1, 1)

    # 指定结束日期
    end_date = datetime(end_year + 1, 1, 1)

    # 循环自增日期
    while current_date < end_date:
        # 转为8位字符串格式 YYYYMMDD
        date_str = current_date.strftime('%Y%m%d')
        nc_file = 'E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_' + date_str + '.nc'
        # data = xr.open_dataset(nc_file)
        clip_nc_region(nc_file, date_str)
        print(nc_file)
        # 日期自增1天
        current_date += timedelta(days=1)




E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950101.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950102.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950103.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950104.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950105.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950106.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950107.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950108.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950109.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950110.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950111.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950112.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950113.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950114.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950115.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950116.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950117.nc
E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19950118.nc
E:/DataSet/redos/REDOS_1.0_1

In [13]:
#对比原数据与截取后的数据
original_data = xr.open_dataset('E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19951231.nc')
# 读取截取后的数据
subset_data = xr.open_dataset('E:/DataSet/redos/Subset_1.0_1995/subset_19951231.nc')
lon_min, lon_max = 112, 117.1
lat_min, lat_max = 12.9, 15.7
original_subset_t = original_data['t'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
original_subset_s = original_data['s'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min,lat_max ))
# original_subset_u = original_data['u'].sel(lonu=slice(lon_min, lon_max), latu=slice(lat_min, lat_max))
# original_subset_v = original_data['v'].sel(lonv=slice(lon_min, lon_max), latv=slice(lat_min, lat_max))
t_equal = original_subset_t.equals(subset_data['t'])
s_equal = original_subset_s.equals(subset_data['s'])
# u_equal = original_subset_u.equals(subset_data['u'])
# v_equal = original_subset_v.equals(subset_data['v'])
# print(original_subset_u[:5, :5]) 
subset_u = subset_data['u']
print(subset_u[:5, :5]) 
print(original_data['u'][:5, :5])
# print(f"t data match: {t_equal}")
# print(f"s data match: {s_equal}")
# print(f"u data match: {u_equal}")
# print(f"v data match: {v_equal}")


<xarray.DataArray 'u' (lv: 5, latu: 5, lonu: 52)>
[1300 values with dtype=float64]
Coordinates:
  * lonu     (lonu) float64 112.0 112.1 112.2 112.3 ... 116.8 116.9 117.0 117.1
  * latu     (latu) float64 12.99 13.09 13.18 13.28 13.38
Dimensions without coordinates: lv
<xarray.DataArray 'u' (lv: 5, latu: 5, lonu: 350)>
[8750 values with dtype=float32]
Dimensions without coordinates: lv, latu, lonu
Attributes:
    long_name:  u component of current
    units:      m/s


((24, 28, 52), (24, 28, 52), (28, 52), (24, 28, 52), (24, 28, 52))

In [8]:
import xarray as xr
import numpy as np
import netCDF4 as nc
# 打开保存的 NetCDF 文件
# subset_data = xr.open_dataset('subset_file_3.nc')
# t=subset_data['t']
# s=subset_data['s']
# zeta=subset_data['zeta']
# u=subset_data['u']
# v=subset_data['v']
original_data = xr.open_dataset('E:/DataSet/redos/REDOS_1.0_1995/REDOS_1.0_19951231.nc')
file_obj = nc.Dataset('E:/DataSet/redos/Subset_1.0_1995/subset_19951231.nc')
t = file_obj.variables['t']
t_data=original_data['t']
t_arr=t[:]
# 定义经纬度范围
lon_min, lon_max = 112, 117.1
lat_min, lat_max = 12.9, 15.7

# 选择该区域内的 t 变量数据
subset_t = original_data['t'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

# 输出该区域内的 t 数据
print(subset_t)
print(subset_t.values)  # 输出具体的数值
# t_arr[0]
def scaler(data):
    #normalise [0,1]
    data_max = np.nanmax(data)
    data_min = np.nanmin(data)
    data_scale = data_max - data_min
    data_std = (data - data_min) / data_scale
    # data_std = data_std * (2)  -1
    data_std [np.isnan(data_std)] = 0
    print(data_max,data_min,data_std)
    return data_max,data_min,data_scale
scaler(subset_t[0])

<xarray.DataArray 't' (lv: 24, lat: 28, lon: 52)>
[34944 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 112.0 112.1 112.2 112.3 ... 116.8 116.9 117.0 117.1
  * lat      (lat) float64 12.99 13.09 13.18 13.28 ... 15.32 15.41 15.51 15.61
Dimensions without coordinates: lv
Attributes:
    long_name:  temperature
    units:      degree_Celsius
[[[ 4.5385175  4.544624   4.544624  ...  4.71996    4.7138543  4.70862  ]
  [ 4.5603256  4.570793   4.5786443 ...  4.7121096  4.706003   4.705131 ]
  [ 4.6144094  4.6466856  4.667621  ...  4.704259   4.694663   4.6981525]
  ...
  [ 4.6170263  4.6353455  4.61092   ...  4.4443064  4.4765825  4.526305 ]
  [ 4.5952187  4.6144094  4.6213884 ...  4.452158   4.4931564  4.5367727]
  [ 4.5463686  4.596091   4.6423235 ...  4.4678593  4.507986   4.5489855]]

 [[ 4.7443857  4.756598   4.7609596 ...  5.119484   5.1648445  5.2459707]
  [ 4.8106823  4.829873   4.84383   ...  5.0906973  5.1203566  5.1857805]
  [ 4.9101267  4.9450197  4.9615936 ...

IndexError: 2-dimensional boolean indexing is not supported. 

In [68]:
def tempfunc(nc_file,type,depth):
    daily_data = []
    #zeta为海表没有深度，所以永远为0
    if type=='zeta':
        temp_lv0 = nc_file.variables[type][ :, :]
    else:
        temp_lv0 = nc_file.variables[type][depth, :, :]
    #模拟20天的数据
    for i in range(30):
        daily_data.append(temp_lv0)
    day_lon_lat = np.array(daily_data)
    return day_lon_lat

In [69]:
def create_dataset(data, time_step):
    dataX = []
    for i in range(data.shape[0] - time_step + 1):
        dataX.append(data[i:i + time_step])
    return np.array(dataX)
def read_raw_data(vtype, depth, time_step,nc_file):
    #训练用的数据是第0层，也就是海表，原来那个是按照深度进行划分的，这个nc文件是按天数进行划分的，这里只有一天，所以shape[0]=1
    train_argo = tempfunc(nc_file,vtype,0)
    label_argo = tempfunc(nc_file,vtype,depth)
    width = train_argo.shape[2] #对应经度
    lenth = train_argo.shape[1] #对应纬度
    X = create_dataset(train_argo, time_step)
    X = X.reshape(X.shape[0],time_step,lenth,width,1)
    Y = label_argo[time_step-1 : label_argo.shape[0]] 
    Y =Y.reshape(Y.shape[0],lenth,width,1)
    #X 转置维度，变为 (样本数, 时间步长, 通道数, 纬度, 经度)。
    #Y 转置维度，变为 (样本数, 时间步长， 经度, 纬度)。
    X = X.transpose(0,1,4,2,3)
    Y = Y.transpose(0,3,1,2)
    return X, Y

In [70]:
#这几个数据格式一样，但是内容不一样，读的分别是不同的列
import netCDF4 as nc
file_path = './subset_file_3.nc'
file_obj = nc.Dataset(file_path)
train_sssa,_=read_raw_data('s',0,3,file_obj)
train_ssha,_ = read_raw_data('zeta',0,3,file_obj) #海面高度异常（Sea Level Anomaly）,他写的是sla，但是这里是zeta
train_sswu,_ = read_raw_data('u',0,3,file_obj)#U vwnd分量的风速（即沿经度方向的风速）,这里是u
train_sswv,_ = read_raw_data('v',0,3,file_obj)#V vwnd分量的风速（即沿纬度方向的风速），这里是v
train_argo, label_argo = read_raw_data('t', 1, 3,file_obj)#temp 代表温度数据,预测深度为1时的海温

In [71]:
train_sssa.shape,train_ssha.shape,train_sswu.shape,train_sswv.shape,label_argo.shape,train_argo.shape

((28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 1, 28, 52),
 (28, 3, 1, 28, 52))

In [14]:
import torch
import numpy as np

# 示例 Tensor
tensor = torch.tensor([[[[0.9920, 0.9946, 0.9946, 0.9946, 0.9945, 0.9913],
                         [0.9947, 0.9950, 0.9950, 0.9950, 0.9950, 0.9948],
                         [0.9945, 0.9950, 0.9950, 0.9950, 0.9950, 0.9947]]]])

# 转换为 NumPy 数组
tensor_np = tensor.cpu().numpy()

# 打印 Tensor 和 NumPy 数组的统计特性
print("Tensor dtype:", tensor.dtype)
print("Tensor min/max:", tensor.min().item(), tensor.max().item())
print("NumPy array min/max:", tensor_np.min(), tensor_np.max())


Tensor dtype: torch.float32
Tensor min/max: 0.9912999868392944 0.9950000047683716
NumPy array min/max: 0.9913 0.995


In [1]:
import numpy as np
import torch





NumPy dtype: float32
NumPy array: [0.998  0.999  0.9995 0.9999]
PyTorch dtype: torch.float32
PyTorch Tensor: tensor([0.9980, 0.9990, 0.9995, 0.9999])
Converted back NumPy array: [0.998  0.999  0.9995 0.9999]
